In [1]:
import pandas as pd
data = pd.read_csv("rainfall_data.csv")
data=data.iloc[:,3:]
# calculate 95th percentile for each column
percentiles = data.quantile(0.95)

# filter rows above 95th percentile for each column
filtered_data = data[(data > percentiles).any(axis=1)]




print(filtered_data)

       rainfall
6     24.932436
7     23.246956
41    33.846075
64    23.561225
83    22.738915
...         ...
2534  21.900691
2538  24.230811
2541  22.366934
2551  34.073259
2552  21.649361

[129 rows x 1 columns]


In [2]:
data = pd.read_csv("rainfall_data.csv")
# label extreme rows in original dataframe
data['rainfall_class'] = 'normal'
data.loc[data.index.isin(filtered_data.index), 'rainfall_class'] = 'extreme'


In [3]:
import numpy as np
import copy
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras import backend as K
import os
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import seed
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc



In [6]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

def weighted_loss(sparsity_weight=0.01, sparsity_target=0.001):
    def loss(y_true, y_pred):
        mse_loss = K.mean(K.square(y_true - y_pred), axis=-1)
        kl_divergence = sparsity_weight * K.sum(
            sparsity_target * K.log(sparsity_target / (K.mean(K.clip(abs(y_pred), 1e-10, 1e10)) + 1e-10) + 1e-10),
            axis=-1
        ) + sparsity_weight * K.sum(
            (1 - sparsity_target) * K.log(
                (1 - sparsity_target) / (1 - (K.mean(K.clip(abs(y_pred), 1e-10, 1e10)) + 1e-10)) + 1e-10),
            axis=-1
        )
        return mse_loss + kl_divergence

    return loss

def train_autoencoder(train_data, reg_strength, num_epochs, batch_size):
    # Define the autoencoder architecture
    input_layer = Input(shape=(train_data.shape[1],))
    encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=reg_strength))(input_layer)
    decoded = Dense(train_data.shape[1], activation=None)(encoded)

    # Create the autoencoder
    autoencoder = Model(input_layer, decoded)

    # Compile the autoencoder with the weighted loss function
    autoencoder.compile(optimizer='adam', loss=weighted_loss())

    # Train the model
    history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

    return autoencoder

def calculate_reconstruction_error(test_data, autoencoder):
    # Reconstruct the input data using the autoencoder
    reconstructed_data = autoencoder.predict(test_data)

    # Calculate the RE for each feature
    reconstruction_error = np.abs(test_data - reconstructed_data)

    return reconstruction_error

def determine_thresholds(normal_data, autoencoder):
    # Calculate the reconstruction error for normal data
    normal_reconstruction_error = calculate_reconstruction_error(normal_data, autoencoder)

    # Determine the maximum error for each feature
    max_error = np.max(normal_reconstruction_error, axis=0)

    return max_error

def classify_data(test_data, thresholds, autoencoder):
    # Calculate the reconstruction error for test data
    reconstruction_error = calculate_reconstruction_error(test_data, autoencoder)

    # Compare the RE of each feature to its corresponding threshold
    is_anomalous = np.any(reconstruction_error > thresholds, axis=1)

    return is_anomalous

# Split the data into normal and extreme classes
normal_data = data[data["rainfall_class"] == "normal"]
extreme_data = data[data["rainfall_class"] == "extreme"]

# Convert the data to numpy arrays
train_data = normal_data[["windspeed", "tpw", "rainfall"]].values.astype("float32")
test_data = extreme_data[["windspeed", "tpw", "rainfall"]].values.astype("float32")

# Define the hyperparameters
reg_strength = 0.001
num_epochs = 50
batch_size = 100

# Train the autoencoder using only normal data
autoencoder = train_autoencoder(train_data, reg_strength, num_epochs, batch_size)

# Determine the thresholds for each feature
thresholds = determine_thresholds(train_data, autoencoder)

# Classify the test data
is_anomalous = classify_data(test_data, thresholds, autoencoder)

# Print the classification results
print("Anomalous data points:")
print(extreme_data[is_anomalous])


ValueError: in user code:

    File "C:\Users\shine\anaconda3\envs\environment\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\shine\AppData\Local\Temp\ipykernel_26636\3981596824.py", line 10, in loss  *
        kl_divergence = sparsity_weight * K.sum(
    File "C:\Users\shine\anaconda3\envs\environment\lib\site-packages\keras\backend.py", line 2544, in sum
        return tf.reduce_sum(x, axis, keepdims)

    ValueError: Invalid reduction dimension -1 for input with 0 dimensions. for '{{node loss/Sum}} = Sum[T=DT_FLOAT, Tidx=DT_INT32, keep_dims=false](loss/mul, loss/Sum/reduction_indices)' with input shapes: [], [] and with computed input tensors: input[1] = <-1>.


In [7]:
extreme_data=data[data["rainfall_class"]=="extreme"]
normal_data=data[data["rainfall_class"]=="normal"]

In [8]:
def weighted_loss(sparsity_weight=0.01, sparsity_target=0.001):
    def loss(y_true, y_pred):
        mse_loss = K.mean(K.square(abs(y_true[:,2] - y_pred[:,2])), axis=-1)
        kl_divergence = sparsity_weight * K.sum(sparsity_target * K.log(abs(sparsity_target / K.mean(abs(y_pred[:, 2])))) 
                        + abs((1 - sparsity_target)) * K.log(abs((1 - sparsity_target) / (1 - K.mean(abs(y_pred[:, 2]))))))
        return mse_loss + kl_divergence
    return loss

In [9]:

reg_strengths = [0.1, 1, 10, 100]
num_epochs = 3000
batch_size = 100
#train_data = normal_data[["windspeed_discretized","tpw_discretized"]].values.astype("float32")
train_data = normal_data[["windspeed","tpw","rainfall"]].values.astype("float32")

#best_reg_strength = find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size)

# Define the autoencoder architecture with the best regularization strength
input_layer = Input(shape=(3,),name='input')
encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=20),name='encoder')(input_layer)
decoded = Dense(3, activation=None, name='decoder')(encoded)

# Create the autoencoder
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder with the weighted loss function
#autoencoder.compile(optimizer='adam', loss="mse")
autoencoder.compile(optimizer='adam', loss=weighted_loss())

# Train the model with the majority class
history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)
#history = autoencoder.fit(train_data,epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)


Epoch 1/3000
20/20 [==============================] - 2s 22ms/step - loss: 137.6824 - val_loss: 125.1913
Epoch 2/3000
20/20 [==============================] - 0s 5ms/step - loss: 117.6649 - val_loss: 108.3685
Epoch 3/3000
20/20 [==============================] - 0s 5ms/step - loss: 102.5605 - val_loss: 95.5568
Epoch 4/3000
20/20 [==============================] - 0s 4ms/step - loss: 90.8828 - val_loss: 85.2640
Epoch 5/3000
20/20 [==============================] - 0s 4ms/step - loss: 81.3364 - val_loss: 76.7347
Epoch 6/3000
20/20 [==============================] - 0s 4ms/step - loss: 73.3957 - val_loss: 69.4419
Epoch 7/3000
20/20 [==============================] - 0s 4ms/step - loss: 66.4385 - val_loss: 62.9502
Epoch 8/3000
20/20 [==============================] - 0s 4ms/step - loss: 60.2302 - val_loss: 57.1355
Epoch 9/3000
20/20 [==============================] - 0s 5ms/step - loss: 54.6612 - val_loss: 51.9170
Epoch 10/3000
20/20 [==============================] - 0s 4ms/step - loss: 4

20/20 [==============================] - 0s 4ms/step - loss: 1.7751 - val_loss: 1.7810
Epoch 82/3000
20/20 [==============================] - 0s 4ms/step - loss: 1.7503 - val_loss: 1.7559
Epoch 83/3000
20/20 [==============================] - 0s 4ms/step - loss: 1.7258 - val_loss: 1.7337
Epoch 84/3000
20/20 [==============================] - 0s 5ms/step - loss: 1.7026 - val_loss: 1.7070
Epoch 85/3000
20/20 [==============================] - 0s 4ms/step - loss: 1.6784 - val_loss: 1.6859
Epoch 86/3000
20/20 [==============================] - 0s 5ms/step - loss: 1.6568 - val_loss: 1.6615
Epoch 87/3000
20/20 [==============================] - 0s 4ms/step - loss: 1.6356 - val_loss: 1.6392
Epoch 88/3000
20/20 [==============================] - 0s 4ms/step - loss: 1.6138 - val_loss: 1.6212
Epoch 89/3000
20/20 [==============================] - 0s 4ms/step - loss: 1.5941 - val_loss: 1.5971
Epoch 90/3000
20/20 [==============================] - 0s 4ms/step - loss: 1.5718 - val_loss: 1.5757
Epoc

20/20 [==============================] - 0s 5ms/step - loss: 0.6783 - val_loss: 0.6756
Epoch 162/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.6712 - val_loss: 0.6710
Epoch 163/3000
20/20 [==============================] - 0s 5ms/step - loss: 0.6656 - val_loss: 0.6655
Epoch 164/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.6613 - val_loss: 0.6624
Epoch 165/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.6564 - val_loss: 0.6565
Epoch 166/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.6509 - val_loss: 0.6506
Epoch 167/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.6464 - val_loss: 0.6466
Epoch 168/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.6423 - val_loss: 0.6456
Epoch 169/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.6387 - val_loss: 0.6373
Epoch 170/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.6327 - val_loss: 0.

20/20 [==============================] - 0s 4ms/step - loss: 0.3747 - val_loss: 0.3732
Epoch 242/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3711 - val_loss: 0.3710
Epoch 243/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3701 - val_loss: 0.3697
Epoch 244/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3667 - val_loss: 0.3692
Epoch 245/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3653 - val_loss: 0.3698
Epoch 246/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3638 - val_loss: 0.3645
Epoch 247/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3649 - val_loss: 0.3673
Epoch 248/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3604 - val_loss: 0.3590
Epoch 249/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3565 - val_loss: 0.3605
Epoch 250/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.3565 - val_loss: 0.

20/20 [==============================] - 0s 4ms/step - loss: 0.2549 - val_loss: 0.2545
Epoch 322/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.2532 - val_loss: 0.2542
Epoch 323/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.2523 - val_loss: 0.2529
Epoch 324/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.2510 - val_loss: 0.2510
Epoch 325/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.2496 - val_loss: 0.2501
Epoch 326/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.2492 - val_loss: 0.2504
Epoch 327/3000
20/20 [==============================] - 0s 5ms/step - loss: 0.2486 - val_loss: 0.2490
Epoch 328/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.2469 - val_loss: 0.2470
Epoch 329/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.2460 - val_loss: 0.2459
Epoch 330/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.2454 - val_loss: 0.

...

20/20 [==============================] - 0s 4ms/step - loss: 0.0215 - val_loss: 0.0220
Epoch 2622/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0216 - val_loss: 0.0236
Epoch 2623/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0228 - val_loss: 0.0228
Epoch 2624/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0219 - val_loss: 0.0218
Epoch 2625/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0215 - val_loss: 0.0218
Epoch 2626/3000
20/20 [==============================] - 0s 6ms/step - loss: 0.0215 - val_loss: 0.0219
Epoch 2627/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0214 - val_loss: 0.0222
Epoch 2628/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0215 - val_loss: 0.0227
Epoch 2629/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0221 - val_loss: 0.0218
Epoch 2630/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0235 - val

20/20 [==============================] - 0s 4ms/step - loss: 0.0211 - val_loss: 0.0217
Epoch 2701/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0208 - val_loss: 0.0212
Epoch 2702/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0214 - val_loss: 0.0244
Epoch 2703/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0221 - val_loss: 0.0222
Epoch 2704/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0209 - val_loss: 0.0212
Epoch 2705/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0208 - val_loss: 0.0211
Epoch 2706/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0209 - val_loss: 0.0220
Epoch 2707/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0213 - val_loss: 0.0243
Epoch 2708/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0234 - val_loss: 0.0217
Epoch 2709/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0210 - val

20/20 [==============================] - 0s 4ms/step - loss: 0.0206 - val_loss: 0.0233
Epoch 2780/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0220 - val_loss: 0.0234
Epoch 2781/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0213 - val_loss: 0.0225
Epoch 2782/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0256 - val_loss: 0.0385
Epoch 2783/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0422 - val_loss: 0.0253
Epoch 2784/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0210 - val_loss: 0.0207
Epoch 2785/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0202 - val_loss: 0.0205
Epoch 2786/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0201 - val_loss: 0.0215
Epoch 2787/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0204 - val_loss: 0.0213
Epoch 2788/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0203 - val

20/20 [==============================] - 0s 4ms/step - loss: 0.0433 - val_loss: 0.0280
Epoch 2859/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0223 - val_loss: 0.0215
Epoch 2860/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0201 - val_loss: 0.0204
Epoch 2861/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0196 - val_loss: 0.0199
Epoch 2862/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0195 - val_loss: 0.0200
Epoch 2863/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0195 - val_loss: 0.0199
Epoch 2864/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0196 - val_loss: 0.0199
Epoch 2865/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0195 - val_loss: 0.0201
Epoch 2866/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0194 - val_loss: 0.0199
Epoch 2867/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0194 - val

20/20 [==============================] - 0s 4ms/step - loss: 0.0323 - val_loss: 0.0294
Epoch 2938/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0251 - val_loss: 0.0207
Epoch 2939/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0210 - val_loss: 0.0193
Epoch 2940/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0190 - val_loss: 0.0193
Epoch 2941/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0188 - val_loss: 0.0193
Epoch 2942/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0189 - val_loss: 0.0193
Epoch 2943/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0190 - val_loss: 0.0194
Epoch 2944/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0188 - val_loss: 0.0193
Epoch 2945/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0188 - val_loss: 0.0193
Epoch 2946/3000
20/20 [==============================] - 0s 4ms/step - loss: 0.0188 - val

In [10]:
from tensorflow.keras.models import load_model

# Save the model to a file
autoencoder.save("my_autoencoder_model.h5")


In [11]:
#test_data = data[["windspeed_discretized","tpw_discretized","rainfall_discretized"]].values
test_data = data[["windspeed","tpw","rainfall"]].values
autoencoder = load_model("my_autoencoder_model.h5", custom_objects={"loss": weighted_loss()})

predicted_data = autoencoder.predict(test_data)

81/81 [==============================] - 0s 1ms/step


In [14]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

def calculate_reconstruction_error(test_data, autoencoder):
    # Reconstruct the input data using the autoencoder
    reconstructed_data = autoencoder.predict(test_data)

    # Calculate the reconstruction error for each feature
    reconstruction_error = np.abs(test_data - reconstructed_data)

    return reconstruction_error


# Load the trained autoencoder model
autoencoder = load_model("my_autoencoder_model.h5", custom_objects={"loss": weighted_loss()})

# Get the encoded representation of the input data
encoded_X = autoencoder.predict(test_data)

# Calculate the reconstruction error for each feature
reconstruction_error = calculate_reconstruction_error(test_data, autoencoder)

# Determine the thresholds for each feature
thresholds = np.max(reconstruction_error, axis=0)

# Classify data points as normal or anomalous based on the reconstruction error and thresholds
is_normal = np.all(reconstruction_error <= thresholds, axis=1)
is_anomalous = np.logical_not(is_normal)

# Print the classification result for each data point
for i in range(len(test_data)):
    if is_normal[i]:
        print('Data point', i+1, 'is classified as normal.')
    else:
        print('Data point', i+1, 'is classified as anomalous.')


81/81 [==============================] - 0s 1ms/step
Data point 1 is classified as normal.
Data point 2 is classified as normal.
Data point 3 is classified as normal.
Data point 4 is classified as normal.
Data point 5 is classified as normal.
Data point 6 is classified as normal.
Data point 7 is classified as normal.
Data point 8 is classified as normal.
Data point 9 is classified as normal.
Data point 10 is classified as normal.
Data point 11 is classified as normal.
Data point 12 is classified as normal.
Data point 13 is classified as normal.
Data point 14 is classified as normal.
Data point 15 is classified as normal.
Data point 16 is classified as normal.
Data point 17 is classified as normal.
Data point 18 is classified as normal.
Data point 19 is classified as normal.
Data point 20 is classified as normal.
Data point 21 is classified as normal.
Data point 22 is classified as normal.
Data point 23 is classified as normal.
Data point 24 is classified as normal.
Data point 25 is cla

In [21]:
reconstruction_error

array([[12.5987642 ,  0.29285112,  0.07608257],
       [13.26999878,  0.46293299,  0.09565068],
       [13.30518609,  0.32249838,  0.08477133],
       ...,
       [ 4.78251903,  0.19117153,  0.03127928],
       [ 4.57780748,  0.20073985,  0.0312258 ],
       [ 5.97717337,  0.45275615,  0.06373571]])

In [22]:
data["reconstruction_error"]=np.sum(reconstruction_error,axis=1)


In [23]:
data

,date,windspeed,tpw,rainfall,rainfall_class,reconstruction_error
0,6/1/2000,12.245595,0.033285,4.688589,normal,12.967698
1,6/2/2000,12.825491,0.044874,11.330589,normal,13.828582
2,6/3/2000,12.921664,0.010243,6.894713,normal,13.712456
3,6/4/2000,15.149001,0.036881,4.613324,normal,15.886046
4,6/5/2000,18.495907,0.139491,8.111635,normal,19.492984
...,...,...,...,...,...,...
2557,9/26/2020,5.577215,0.009963,4.467977,normal,6.234072
2558,9/27/2020,5.184293,0.002341,1.572182,normal,5.715370
2559,9/28/2020,4.469007,0.000867,1.819019,normal,5.004970
2560,9/29/2020,4.259090,0.001416,2.198017,normal,4.809773
